In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

C:\Program Files (x86)\software\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# 生成整型数据的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串型的属性
def _byte_feature(value):
    return tf.train.Feature(byte_list=tf.train.BytesList(value=[value]))

mnist=input_data.read_data_sets('MNIST_data/',dtype=tf.uint8,one_hot=True)
images=mnist.train.images
labels=mnist.train.labels

# 训练数据的图像分辨率
pixels=images.shape[1]
print('pixels',pixels)
num_examples=mnist.train.num_examples

# 输出TFRecords的地址
tfrecord_path='save_tfrecords'
# 创建一个writer来写TFRecord文件
writer=tf.python_io.TFRecordWriter(tfrecord_path)
for index in range(num_examples):
#     将图像矩阵转化成一个字符串
#     image_raw=tf.string(images[index])
#     ss=_byte_feature(image_raw)
#     print('ss:',ss)
#     将一个样例转入到Example Protocal Buffer，并将所有的信息写入这个数据结构
    example=tf.train.Example(features=tf.train.Features(feature={
        'pixels':_int64_feature(pixels),
        'label':_int64_feature(np.argmax(labels[index])),
#         'image_raw':_byte_feature(image_raw)
    }))
    
#     将一个Example写入TFRecord文件
    writer.write(example.SerializeToString())
writer.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
pixels 784


In [5]:
# 读取TFRecord
import tensorflow as tf

# 创建一个reader来读取TFRecord文件中的样例
reader=tf.TFRecordReader()
# 创建一个队列来维护输入文件列表，
filename_queue=tf.train.string_input_producer(['save_tfrecords'])

# 从文件中读取一个样例，也可以使用read_tup_to函数一次读取多个样例
_,serialized_example=reader.read(filename_queue)

# 解析读入的一个样例，如果需要解析多个样例，可以用parse_example函数
features=tf.parse_single_example(serialized_example,
                                features={
                                  'pixels':tf.FixedLenFeature([],tf.int64),
                                    'label':tf.FixedLenFeature([],tf.int64)
                                })

# tf.decode_raw可以将字符串解析成图相对于的像素组
# image=tf.decode_raw(features[''])
label=tf.cast(features['label'],tf.int32)
pixels=tf.cast(features['pixels'],tf.int32)

sess=tf.Session()
# 启动多线程处理数据
coord=tf.train.Coordinator()
threads=tf.train.start_queue_runners(sess=sess,coord=coord)

# 每次运行可以读取TFRecord文件中的一个样例，当所有样例都读完以后，在此样例中会重新读取
for i in range(10):
    print(sess.run([label,pixels]))

[7, 784]
[3, 784]
[4, 784]
[6, 784]
[1, 784]
[8, 784]
[1, 784]
[0, 784]
[9, 784]
[8, 784]
